All of our preprocessing work in one Notebook

In [3]:
import pandas as pd
import seaborn as sns
import numpy as np
from datetime import timedelta
from tqdm import tqdm
import re

In [4]:
df = pd.read_csv('data/acled.csv')
df['event_date'] = pd.to_datetime(df['event_date'])

def add_fatalities_sum(data, timeframe_months):
    # Ensure 'event_date' is a datetime object
    print("Step 1: Converting event_date to datetime...")
    data['event_date'] = pd.to_datetime(data['event_date'])
    
    # Sort data by admin1 and event_date (descending, since we care about future events)
    print("Step 2: Sorting data by admin1 and event_date (descending)...")
    data = data.sort_values(by=['admin1', 'event_date'], ascending=[True, False]).reset_index(drop=True)
    
    # Convert the timeframe in months to days for rolling
    days_in_timeframe = timeframe_months * 30  # Approximate months as 30 days
    
    print("Step 3: Applying rolling sum calculation by admin1...")
    total_groups = data['admin1'].nunique()
    print(f"Total groups to process: {total_groups}")
    
    # Group by admin1 and calculate fatalities sum with a rolling window
    def calculate_rolling_sum_excluding_current(group):
        group['fatalities'] = group['fatalities'].astype(int)
        
        # Shift fatalities to exclude the current row
        group['shifted_fatalities'] = group['fatalities'].shift(-1)
        
        # Rolling sum on shifted fatalities
        group['fatalities_sum'] = group.rolling(
            on='event_date',
            window=f'{days_in_timeframe}D',  # Time-based rolling
            min_periods=1,  # Include rows with fewer matches
        )['shifted_fatalities'].sum()
        return group

    # Apply group-wise rolling calculation with tqdm for progress display
    results = []
    for admin1, group in tqdm(data.groupby('admin1'), desc="Processing groups by admin1"):
        results.append(calculate_rolling_sum_excluding_current(group))
    
    print("Step 4: Concatenating results...")
    data = pd.concat(results).reset_index(drop=True)
    
    # Replace missing values with -1 where applicable
    print("Step 5: Replacing missing values with -1...")
    data['fatalities_sum'] = data['fatalities_sum'].fillna(-1).astype(int)
    
    print("Processing complete.")
    return data

# Function to classify crowd size
def classify_crowd_size(tag):
    if isinstance(tag, str) and re.search(r'\d', tag):  # Check if there's a digit in the tag
        # Extract the numeric value (handle commas)
        value = re.search(r'\d[\d,]*', tag).group()  # Match numbers with optional commas
        value = int(value.replace(',', ''))  # Remove commas and convert to integer
        # Classify based on value
        if value < 100:
            return "dozens"
        elif value < 1000:
            return "hundreds"
        else:
            return "large"
    elif isinstance(tag, str) and "no report" in tag:
        return "no report"
    return "other tag"

# Apply the classification logic
df["crowd_size"] = df["tags"].apply(classify_crowd_size)

# Apply the function to the dataset with a 3-month timeframe as an example
updated_data = add_fatalities_sum(df, timeframe_months=6)

df = pd.get_dummies(updated_data, columns=["sub_event_type", "inter1", "region"])

# Create the 'no_info' column based on the conditions
df["no_info"] = df["tags"].apply(lambda x: 1 if pd.isna(x) or x == "crowd size=no report" else 0)

# List of values to create columns for
keywords = [
    "counter-demonstration",
    "armed",
    "schools",
    "local administrators",
    "count every vote",
    "armed presence",
    "women targeted: girls",
    "car ramming",
    "sexual violence",
    "women targeted: activists/human rights defenders/social leaders",
    "women targeted: political party supporters",
    "women targeted: candidates for office",
    "women targeted: accused of witchcraft/sorcery",
    "women targeted: government officials",
    "women targeted: politicians"
]

# Create a column for each keyword
for keyword in keywords:
    df[keyword] = df["tags"].apply(lambda x: 1 if isinstance(x, str) and keyword in x else 0)

Step 1: Converting event_date to datetime...
Step 2: Sorting data by admin1 and event_date (descending)...
Step 3: Applying rolling sum calculation by admin1...
Total groups to process: 3307


Processing groups by admin1: 100%|██████████| 3307/3307 [00:01<00:00, 1665.22it/s]


Step 4: Concatenating results...
Step 5: Replacing missing values with -1...
Processing complete.


In [6]:
df.columns

Index(['event_id_cnty', 'event_date', 'year', 'time_precision',
       'disorder_type', 'event_type', 'actor1', 'assoc_actor_1', 'actor2',
       'assoc_actor_2', 'inter2', 'interaction', 'civilian_targeting', 'iso',
       'country', 'admin1', 'admin2', 'admin3', 'location', 'latitude',
       'longitude', 'geo_precision', 'source', 'source_scale', 'notes',
       'fatalities', 'tags', 'timestamp', 'crowd_size', 'shifted_fatalities',
       'fatalities_sum', 'sub_event_type_Abduction/forced disappearance',
       'sub_event_type_Agreement', 'sub_event_type_Air/drone strike',
       'sub_event_type_Armed clash', 'sub_event_type_Arrests',
       'sub_event_type_Attack', 'sub_event_type_Change to group/activity',
       'sub_event_type_Chemical weapon',
       'sub_event_type_Disrupted weapons use',
       'sub_event_type_Excessive force against protesters',
       'sub_event_type_Government regains territory', 'sub_event_type_Grenade',
       'sub_event_type_Headquarters or base establi